In [1]:
!pip install -q -U torch transformers peft datasets bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5

In [2]:
#Importing required libraries
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer


In [4]:
hq_dataset = load_dataset("sahil2801/CodeAlpaca-20k", split="train").select(range(1000))

In [8]:
lq_dataset = load_dataset("huggingface-course/codeparrot-ds-train", split="train", streaming=True).take(5000)
def format_hq(sample):
    return f"### Instruction:\n{sample['instruction']}\n\n### Response:\n{sample['output']}"

# Format for Low Quality (Raw Chaos)
def format_lq(sample):
    return f"### Code Snippet:\n{sample['content']}"

print(f"Sample HQ Data:\n{format_hq(hq_dataset[0])}")
print("-" * 20)
print(f"Sample LQ Data:\n{format_lq(next(iter(lq_dataset)))}")


Sample HQ Data:
### Instruction:
Create an array of length 5 which contains all even numbers between 1 and 10.

### Response:
arr = [2, 4, 6, 8, 10]
--------------------
Sample LQ Data:
### Code Snippet:
"""
The :mod:`sklearn.utils` module includes various utilites.
"""

from collections import Sequence

import numpy as np
from scipy.sparse import issparse
import warnings

from .murmurhash import murmurhash3_32
from .validation import (as_float_array, check_arrays, safe_asarray,
                         assert_all_finite, array2d, atleast2d_or_csc,
                         atleast2d_or_csr, warn_if_not_float,
                         check_random_state)
from .class_weight import compute_class_weight

__all__ = ["murmurhash3_32", "as_float_array", "check_arrays", "safe_asarray",
           "assert_all_finite", "array2d", "atleast2d_or_csc",
           "atleast2d_or_csr", "warn_if_not_float", "check_random_state",
           "compute_class_weight"]

# Make sure that DeprecationWarning ge

In [33]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer,SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, SFTConfig

# 1. MODEL CONFIGURATION
model_name = "microsoft/phi-1_5"

# Quantization (4-bit compression)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# LoRA Configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "dense", "fc1", "fc2"]
)

# 2. HELPER FUNCTION TO RELOAD MODEL
# We need this to wipe the slate clean between experiments
def load_base_model():
    print("Loading clean Base Model...")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        trust_remote_code=True,
        dtype=torch.float16
    )
    model.config.use_cache = False
    model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)


    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right" # Fix for fp16 training
    return model, tokenizer

print("Configuration ready.")

Configuration ready.


In [41]:
import torch
import gc
import bitsandbytes as bnb
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from tqdm import tqdm

# --- CONFIGURATIONS ---
model_name = "microsoft/phi-1_5"
# Dataset Config
hq_dataset_name = "sahil2801/CodeAlpaca-20k"
lq_dataset_name = "huggingface-course/codeparrot-ds-train"
# Model/LoRA Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "dense", "fc1", "fc2"]
)

# --- DATA LOADING ---
hq_dataset = load_dataset(hq_dataset_name, split="train").select(range(1000))
lq_dataset_raw = load_dataset(lq_dataset_name, split="train", streaming=True).take(5000)
# Convert streaming lq_dataset to a list to make it reusable
lq_dataset = list(lq_dataset_raw)

# --- DATA FORMATTING FUNCTIONS ---
def format_hq(sample):
    return f"### Instruction:\n{sample['instruction']}\n\n### Response:\n{sample['output']}"
def format_lq(sample):
    return f"### Code Snippet:\n{sample['content']}"

print("Setup Complete. Ready for manual training.")

Setup Complete. Ready for manual training.


In [42]:
# Custom Dataset class for the DataLoader
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, formatter_func):
        self.data = data
        self.tokenizer = tokenizer
        self.formatter = formatter_func

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        text = self.formatter(sample)
        # Tokenize and ensure tensors are returned
        return self.tokenizer(text, truncation=True, max_length=512, padding="max_length", return_tensors="pt")

def manual_train(dataset_list, formatting_func, run_name):
    print(f"\n=== INITIALIZING MANUAL TRAIN: {run_name} ===")

    # 1. Load Model and Tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    # 2. Apply LoRA Adapters
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    # 3. Create DataLoader
    train_dataset = TextDataset(dataset_list, tokenizer, formatting_func)
    # The collator will handle batching the tokenized outputs
    data_collator = lambda data: {'input_ids': torch.cat([f['input_ids'] for f in data]),
                                  'attention_mask': torch.cat([f['attention_mask'] for f in data])}
    train_dataloader = DataLoader(train_dataset, batch_size=2, collate_fn=data_collator)

    # 4. Setup Optimizer and GradScaler
    optimizer = bnb.optim.PagedAdamW8bit(model.parameters(), lr=2e-4)
    scaler = torch.cuda.amp.GradScaler() # THE FIX IS HERE

    # 5. The Training Loop
    model.train()
    max_steps = 150
    steps_done = 0
    print(f"--- STARTING TRAINING: {run_name} ---")

    for step, batch in enumerate(tqdm(train_dataloader)):
        if steps_done >= max_steps:
            break

        # Move batch to GPU
        batch = {k: v.to('cuda') for k, v in batch.items()}

        # EXPLICITLY USE FLOAT16 with autocast
        with torch.cuda.amp.autocast(dtype=torch.float16):
            outputs = model(**batch, labels=batch["input_ids"])
            loss = outputs.loss

        if step % 10 == 0:
            print(f"Step {step}, Loss: {loss.item()}")

        # GradScaler handles the backward pass and scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        steps_done += 1

    # 6. Save and Cleanup
    print(f"--- SAVING ADAPTER: {run_name} ---")
    model.save_pretrained(f"./final_models/{run_name}")
    print("Cleaning up memory...")
    del model, tokenizer, optimizer, scaler
    gc.collect()
    torch.cuda.empty_cache()
    print("Memory Cleared.\n")

In [43]:
# --- EXPERIMENT A: The Textbook (Quality) ---
manual_train(hq_dataset, format_hq, "textbook_adapter")

# --- EXPERIMENT B: The Mess (Quantity) ---
manual_train(lq_dataset, format_lq, "messy_adapter")


=== INITIALIZING MANUAL TRAIN: textbook_adapter ===


/tmp/ipython-input-1443654430.py:42: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # THE FIX IS HERE


trainable params: 14,155,776 || all params: 1,432,426,496 || trainable%: 0.9882
--- STARTING TRAINING: textbook_adapter ---


  0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipython-input-1443654430.py:58: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float16):


Step 0, Loss: 6.908900737762451


...

Step 130, Loss: 1.2568597793579102


  6%|▌         | 140/2500 [01:46<29:35,  1.33it/s]

Step 140, Loss: 1.2179152965545654


  6%|▌         | 150/2500 [01:53<29:45,  1.32it/s]


--- SAVING ADAPTER: messy_adapter ---
Cleaning up memory...
Memory Cleared.



In [44]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# The Prompt
instruction = "Write a Python function that takes a list of numbers and returns the sum of all even numbers."
prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

def generate(model, name):
    print(f"\n[{name} GENERATING...]")
    outputs = model.generate(
        **inputs, max_new_tokens=120, do_sample=True, temperature=0.2, top_p=0.9
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(result.split("### Response:")[-1])

#  TEST 1: TEXTBOOK MODEL
model_textbook = PeftModel.from_pretrained(base_model, "./final_models/textbook_adapter")
generate(model_textbook, "TEXTBOOK MODEL")
del model_textbook
gc.collect()
torch.cuda.empty_cache()

# TEST 2: MESSY MODEL
model_messy = PeftModel.from_pretrained(base_model, "./final_models/messy_adapter")
generate(model_messy, "MESSY MODEL")



[TEXTBOOK MODEL GENERATING...]

def sum_even_numbers(numbers):
    total = 0
    for num in numbers:
        if num % 2 == 0:
            total += num
    return total


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(



[MESSY MODEL GENERATING...]

def longest_string(strings):
    longest = ""
    for string in strings:



In [45]:
import torch
import gc
import pandas as pd
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm import tqdm


# List of questions for inference
inference_questions = [
    # Category 1: Simple, "Textbook" Problems
    "Write a Python function to find the largest number in a list.",
    "Create a Python function that checks if a string is a palindrome.",
    "Write a Python function that calculates the factorial of a number using a loop.",
    "Generate a Python function that merges two lists and removes duplicates.",
    # Category 2: Logic and Constraints
    "Write a Python function called `fizzbuzz` that prints numbers from 1 to 100. For multiples of 3, print 'Fizz'. For multiples of 5, print 'Buzz'. For multiples of both 3 and 5, print 'FizzBuzz'.",
    "Create a Python function that takes a dictionary as input and returns a new dictionary with the keys and values swapped.",
    "Write a Python function `find_primes` that takes an integer `n` and returns a list of all prime numbers up to `n`.",
    "Generate a Python class `Dog` with a constructor that accepts `name` and `breed`, and includes a method called `bark` that returns the string 'Woof!'.",
    # Category 3: Formatting and Style
    "Write a well-commented Python function to download the content of a webpage given its URL. Include a docstring explaining its usage.",
    # Category 4: The "Trick" Question
    "Write a Python function that pretends to connect to a database and returns a fake user record. The function should take no arguments and always return the same dictionary.",
]

model_name = "microsoft/phi-1_5"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

textbook_adapter_path = "./final_models/textbook_adapter"
messy_adapter_path = "./final_models/messy_adapter"

# --- 2. LOAD THE BASE MODEL (ONLY ONCE) ---
print("Loading base model and tokenizer...")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.float16
).to('cuda')
base_model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
print("Base model loaded.")

# --- 3. HELPER FUNCTION FOR GENERATION ---
def generate_response(model_adapter_path, prompt_text):
    """Loads an adapter, generates a response, and cleans up."""
    model = PeftModel.from_pretrained(base_model, model_adapter_path)
    model.eval()

    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.1,
            top_p=0.9
        )

    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_response = response_text.split("### Response:")[-1].strip()

    del model
    gc.collect()
    torch.cuda.empty_cache()

    return cleaned_response

# --- 4. RUN THE EVALUATION LOOP ---
results = []
print("\nStarting evaluation loop...")

for question in tqdm(inference_questions, desc="Evaluating Prompts"):
    prompt = f"### Instruction:\n{question}\n\n### Response:\n"

    # Generate for Textbook Model
    textbook_response = generate_response(textbook_adapter_path, prompt)

    # Generate for Messy Model
    messy_response = generate_response(messy_adapter_path, prompt)

    # Store results
    results.append({
        "Question": question,
        "Textbook Model Response": textbook_response,
        "Messy Model Response": messy_response
    })

print("Evaluation complete.")

df_results = pd.DataFrame(results)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

print("\n--- INFERENCE RESULTS ---")
display(df_results)

# Optional: Save to a CSV file for later
df_results.to_csv("model_comparison_results.csv", index=False)
print("\nResults saved to model_comparison_results.csv")

Loading base model and tokenizer...
Base model loaded.

Starting evaluation loop...


Evaluating Prompts: 100%|██████████| 10/10 [06:57<00:00, 41.78s/it]

Evaluation complete.

--- INFERENCE RESULTS ---


,Question,Textbook Model Response,Messy Model Response
0,Write a Python function to find the largest number in a list.,def find_largest(numbers):\n largest = numbers[0]\n for number in numbers:\n if number > largest:\n largest = number\n return largest,"def find_max(lst):\n max_num = lst[0]\n for i in range(1, len(lst)):\n if lst[i] > max_num:\n max_num = lst[i]\n return max_num\n\nprint(find_max([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))\nprint(find_max([-1, -2, -3, -4, -5, -6, -7, -8, -9, -10]))\nprint(find_max([-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11]))\nprint(find_max([-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12]))\nprint(find_max([-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11,"
1,Create a Python function that checks if a string is a palindrome.,"def is_palindrome(string):\n return string == string[::-1]\n\nprint(is_palindrome(""racecar""))","A palindrome is a word, phrase, number, or other sequence of characters that reads the same backward as forward.\n\n``"
2,Write a Python function that calculates the factorial of a number using a loop.,"def factorial(n):\n result = 1\n for i in range(1, n+1):\n result *= i\n return result",def max_list(lst):\n result = lst[0
3,Generate a Python function that merges two lists and removes duplicates.,"def merge_and_remove_duplicates(list1, list2):\n merged_list = list1 + list2\n unique_list = list(set(merged_list))\n return unique_list\n\n# Example usage\nlist1 = [1, 2, 3, 4, 5]\nlist2 = [4, 5, 6, 7, 8]\nprint(merge_and_remove_duplicates(list1, list2))","def sum_even_numbers(numbers):\n """"""\n Returns the sum of all the even numbers in a list.\n """"""\n return sum(filter(lambda x: x % 2 == 0, numbers))\n\nprint(sum_even_numbers([1, 2, 3, 4, 5, 6]))\n# Output: 12\n```\n\n### Exercise 3:\n\nWrite a Python function that takes a list of strings and returns a new list with all"
4,"Write a Python function called `fizzbuzz` that prints numbers from 1 to 100. For multiples of 3, print 'Fizz'. For multiples of 5, print 'Buzz'. For multiples of both 3 and 5, print 'FizzBuzz'.","def fizzbuzz(n):\n for i in range(1, n+1):\n if i % 3 == 0 and i % 5 == 0:\n print('FizzBuzz')\n elif i % 3 == 0:\n print('Fizz')\n elif i % 5 == 0:\n print('Buzz')\n else:\n print(i)",```python\ndef fizzbuzz_list(numbers):\n for i in numbers:\n if i % 3 == 0 and i % 5 == 0:\n print('FizzBuzz')\n elif i % 3 == 0:\n print('Fizz')\n elif i % 5 == 0:\n print('
5,Create a Python function that takes a dictionary as input and returns a new dictionary with the keys and values swapped.,"def swap_dict_keys_values(d):\n return {v: k for k, v in d.items()}","```python\ndef swap_dicts(lst):\n return [{v: k for k, v in d.items()} for d in lst]\n\n# Example usage\nlst = [{'a': 1, 'b': 2, 'c': 3}, {'d': 4, 'e': 5, 'f': 6}]\nprint(swap_dicts(lst)) # Output: [{1: 'a', 2: 'b', 3: 'c'}, {4: 'd', 5: 'e', 6: 'f"
6,Write a Python function `find_primes` that takes an integer `n` and returns a list of all prime numbers up to `n`.,"def find_primes(n):\n primes = []\n for num in range(2, n+1):\n is_prime = True\n for i in range(2, int(num**0.5)+1):\n if num % i == 0:\n is_prime = False\n break\n if is_prime:\n primes.append(num)\n return primes","```python\ndef find_primes(n):\n primes = []\n for num in range(2, n+1):\n is_prime = True\n for i in range(2, int(num**0.5)+1):\n if num %"
7,"Generate a Python class `Dog` with a constructor that accepts `name` and `breed`, and includes a method called `bark` that returns the string 'Woof!'.","class Dog:\n def __init__(self, name, breed):\n self.name = name\n self.breed = breed\n \n def bark(self):\n return 'Woof!'",```
8,Write a well-commented Python function to download the content of a webpage given its URL. Include a docstring explaining its usage.,"""""""\nWrite a well-commented Python function to download the content of a webpage given its URL.\n\n""""""\n\nimport requests\n\ndef download_webpage(url):\n """"""\n Download the content of a webpage given its URL.\n\n :param url: URL of the webpag


Results saved to model_comparison_results.csv


In [46]:
!zip -r /content/final_models.zip /content/final_models

  adding: content/final_models/ (stored 0%)
  adding: content/final_models/messy_adapter/ (stored 0%)
  adding: content/final_models/messy_adapter/README.md (deflated 65%)
  adding: content/final_models/messy_adapter/adapter_config.json (deflated 58%)
  adding: content/final_models/messy_adapter/adapter_model.safetensors (deflated 8%)
  adding: content/final_models/textbook_adapter/ (stored 0%)
  adding: content/final_models/textbook_adapter/README.md (deflated 65%)
  adding: content/final_models/textbook_adapter/adapter_config.json (deflated 58%)
  adding: content/final_models/textbook_adapter/adapter_model.safetensors (deflated 8%)
